<a href="https://colab.research.google.com/github/simulate111/Deep-Learning-in-Human-Language-Technology/blob/main/course_project_Reza_CoP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep learning in Human Language Technology Project (Template)

- Student(s) Name(s): Mohammadreza Akhtari
- Date: October 2024
- Chosen Corpus:
- Contributions (if group project): -

### Corpus information

- Description of the chosen corpus:
- Paper(s) and other published materials related to the corpus:
- Random baseline performance and expected performance for recent machine learned models:

---

## 1. Setup

In [6]:
# Your code to install and import libraries etc. here
!pip install transformers datasets evaluate
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate
import numpy as np
import os

# Disable W&B logging
os.environ["WANDB_DISABLED"] = "true"


---

## 2. Data download, sampling and preprocessing

### 2.1. Download the corpus

In [2]:
# Your code to download the corpus here
dataset = load_dataset("mteb/amazon_reviews_multi", trust_remote_code=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

### 2.2. Sampling and preprocessing

In [3]:
# Your code for any necessary sampling and preprocessing here
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True)

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
encoded_dataset = dataset.map(preprocess_function, batched=True)

# Downsample if necessary
small_train_dataset = encoded_dataset["train"].shuffle(seed=42).select(range(10000))
small_eval_dataset = encoded_dataset["validation"].shuffle(seed=42).select(range(1000))


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/1200000 [00:00<?, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

---

## 3. Machine learning model

### 3.1. Model training

In [7]:
# Your code to train the transformer based model on the training set and evaluate the performance on the validation set here
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=5)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Updated to use eval_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to=None  # Disable W&B logging
)

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,1.301500,1.140018,0.496000
2,1.045400,1.145733,0.499000
3,0.892400,1.207672,0.511000


TrainOutput(global_step=1875, training_loss=1.0241733723958333, metrics={'train_runtime': 1153.534, 'train_samples_per_second': 26.007, 'train_steps_per_second': 1.625, 'total_flos': 2844907358298336.0, 'train_loss': 1.0241733723958333, 'epoch': 3.0})

### 3.2 Hyperparameter optimization

In [8]:
# Define a set of hyperparameters to search over
learning_rates = [1e-5, 3e-5]# 2e-5,
weight_decays = [0.01, 0.1]#0.0,

best_accuracy = 0
best_hyperparams = {}

for lr in learning_rates:
    for wd in weight_decays:
        training_args = TrainingArguments(
            output_dir="./results",
            eval_strategy="epoch",
            learning_rate=lr,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            num_train_epochs=3,
            weight_decay=wd,
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=small_train_dataset,
            eval_dataset=small_eval_dataset,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics,
        )

        trainer.train()
        eval_results = trainer.evaluate()
        accuracy = eval_results["eval_accuracy"]

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_hyperparams = {"learning_rate": lr, "weight_decay": wd}

print(f"Best hyperparameters: {best_hyperparams}")
print(f"Best accuracy: {best_accuracy}")


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,0.648100,1.438384,0.495000
2,0.472300,1.647363,0.485000
3,0.381300,1.758638,0.494000


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,0.287700,2.345274,0.456000
2,0.205600,2.503536,0.470000
3,0.177900,2.640682,0.479000


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,0.357200,2.340635,0.458000
2,0.315300,1.989231,0.489000
3,0.262100,2.340339,0.486000


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,0.398100,2.304458,0.465000
2,0.260300,2.395875,0.467000
3,0.189800,2.834649,0.480000


Best hyperparameters: {'learning_rate': 1e-05, 'weight_decay': 0.01}
Best accuracy: 0.494


### 3.3. Evaluation on test set

In [9]:
# Your code to evaluate the final model on the test set here
test_results = trainer.evaluate(encoded_dataset["test"])
print(test_results)

{'eval_loss': 2.6981821060180664, 'eval_accuracy': 0.5042, 'eval_runtime': 301.5043, 'eval_samples_per_second': 99.501, 'eval_steps_per_second': 6.219, 'epoch': 3.0}


### 3.4. Cross-lingual experiments

In [10]:
# Your code to train and evaluate the cross-lingual model
cross_lingual_results = {}
for lang in ["de", "es", "fr", "ja", "zh"]:
    test_dataset = load_dataset("mteb/amazon_reviews_multi", lang, split="test")
    encoded_test_dataset = test_dataset.map(preprocess_function, batched=True)
    results = trainer.evaluate(encoded_test_dataset)
    cross_lingual_results[lang] = results
print(cross_lingual_results)

0000.parquet:   0%|          | 0.00/31.9M [00:00<?, ?B/s]

de/validation/0000.parquet:   0%|          | 0.00/800k [00:00<?, ?B/s]

de/test/0000.parquet:   0%|          | 0.00/797k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/200000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

...

Generating train split:   0%|          | 0/200000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

{'de': {'eval_loss': 2.549469232559204, 'eval_accuracy': 0.533, 'eval_runtime': 53.1155, 'eval_samples_per_second': 94.135, 'eval_steps_per_second': 5.893, 'epoch': 3.0}, 'es': {'eval_loss': 2.7785866260528564, 'eval_accuracy': 0.5016, 'eval_runtime': 38.8145, 'eval_samples_per_second': 128.818, 'eval_steps_per_second': 8.064, 'epoch': 3.0}, 'fr': {'eval_loss': 2.7225120067596436, 'eval_accuracy': 0.5078, 'eval_runtime': 42.1824, 'eval_samples_per_second': 118.533, 'eval_steps_per_second': 7.42, 'epoch': 3.0}, 'ja': {'eval_loss': 2.7529549598693848, 'eval_accuracy': 0.4788, 'eval_runtime': 75.1933, 'eval_samples_per_second': 66.495, 'eval_steps_per_second': 4.163, 'epoch': 3.0}, 'zh': {'eval_loss': 2.8825671672821045, 'eval_accuracy': 0.459, 'eval_runtime': 48.9485, 'eval_samples_per_second': 102.148, 'eval_steps_per_second': 6.394, 'epoch': 3.0}}


In [11]:
# Train on English and evaluate on English
trainer.train()
baseline_results = trainer.evaluate(encoded_dataset["test"])
print("Baseline results (Train on English, Evaluate on English):", baseline_results)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.411300,2.040717,0.484000
2,0.307000,2.284298,0.478000
3,0.192400,2.983995,0.486000


Baseline results (Train on English, Evaluate on English): {'eval_loss': 2.875011920928955, 'eval_accuracy': 0.5032666666666666, 'eval_runtime': 302.3397, 'eval_samples_per_second': 99.226, 'eval_steps_per_second': 6.202, 'epoch': 3.0}


In [12]:
# Your code to evaluate the final model on the test set here
test_results = trainer.evaluate(encoded_dataset["test"])
print(test_results)


{'eval_loss': 2.875011920928955, 'eval_accuracy': 0.5032666666666666, 'eval_runtime': 300.9303, 'eval_samples_per_second': 99.691, 'eval_steps_per_second': 6.231, 'epoch': 3.0}


In [ ]:
# Train on multiple non-English languages and evaluate on English
non_english_train_dataset = encoded_dataset["train"].filter(lambda example: not example['id'].startswith('en'))
trainer.train_dataset = non_english_train_dataset
trainer.train()
mixed_zero_shot_results = trainer.evaluate(encoded_dataset["test"])
print("Mixed zero-shot results (Train on multiple non-English languages, Evaluate on English):", mixed_zero_shot_results)


Filter:   0%|          | 0/1200000 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss


---

## 4. Results and summary

### 4.1 Corpus insights

(Briefly discuss what you learned about the corpus and its annotation)

### 4.2 Results

(Briefly summarize your results)

### 4.3 Relation to random baseline / expected performance / state of the art

(Compare your results with the random and state-of-the-art performance)

---

## 5. Bonus Task (optional)

### 5.1. and 5.2. Model and Data selection

(Briefly describe which model was used and why. Also, describe how the test data was downsampled, include relevant code.)

### 5.3. Prompt design

(Include your final prompt here. Also, explain here all prompt engineering insights you learned during the project.)

### 5.4. Generate

In [ ]:
# Your code to run the generative model and extract predictions from the model output.

### 5.5. Evaluation and results

(Briefly summarize your results)

### 5.6 Error analysis (group projects only)

(Present the error analysis results here)